In [1]:
import json
import csv
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import Point, MultiPoint, Polygon, MultiPolygon
from shapely.affinity import scale
import matplotlib.pyplot as plt

import glob
import os
import datetime

In [61]:
path = '/Users/margaretmccall/Documents/2020 Spring/CE 295/0 - Final Project/FinalData/'

bids = pd.read_csv(path+'df_as_bid_aggregated_data.csv')
plans = pd.read_csv(path+'df_as_plan.csv')
energy_prices = pd.read_csv(path+'df_energy_price.csv') #this was an error in Nick's; was aggregated data
price_vol = pd.read_csv(path+'df_AS_price_vol.csv')
generation = pd.read_csv(path+'df_generation.csv')
weather = pd.read_csv(path+'df_weather_forecast_ercot.csv')

data = {'bids':bids,
        'plans':plans,
        'energy_prices':energy_prices,
        'price_vol':price_vol,
        'generation':generation,
        'weather':weather,
       }

In [62]:
#should note to nick i dropped duplicates in energy prices, and fixed the weather -1 thing

In [63]:
#Converting date columns to datetime format
for key in ['energy_prices','price_vol','generation']:
    data[key]['dt'] = pd.to_datetime(data[key]['date'] + " " + data[key]['hr_beg'])
    data[key].drop(columns=['date','hr_beg'], inplace=True)
    
for key in ['bids','plans','weather']:
    single_dig_hrs = (data[key]['hr_beg'] >= 0) & (data[key]['hr_beg'] < 10)
    data[key]['time'] = data[key]['hr_beg'].astype(str)
    data[key]['time'] = data[key]['time'] + ":00"
    data[key].loc[single_dig_hrs,'time'] = "0" + data[key].loc[single_dig_hrs,'time']
    
    data[key]['dt'] = pd.to_datetime(data[key]['date'] + " " + data[key]['time'])
    data[key].drop(columns=['date','hr_beg','time'], inplace=True)
    

In [68]:
#Getting starting and ending date to create daterange for merged df
mindate = []
maxdate = []
for key in data.keys():
    mindate.append(min(data[key]['dt']))
    maxdate.append(max(data[key]['dt']))
    
date_range = pd.date_range(min(mindate), max(maxdate),freq='H',)

In [97]:
#Merging dfs
df = pd.DataFrame(date_range, columns=["dt"]) #wanted to just make this an index, but wasn't working w merge

for key in data.keys():
    df = df.merge(data[key], how='left', on='dt', left_index=True)
    df.reset_index(inplace=True,drop=True)

df.set_index('dt')
df.head()

,dt,OFFNS_Unweighted Average Price,OFFNS_Max Price,OFFNS_Min Price,OFFNS_Total Quantity,OFFNS_Number of Bids,OFFNS_Weighted Avg Price,ONNS_Unweighted Average Price,ONNS_Max Price,ONNS_Min Price,ONNS_Total Quantity,ONNS_Number of Bids,ONNS_Weighted Avg Price,REGDN_Unweighted Average Price,REGDN_Max Price,REGDN_Min Price,REGDN_Total Quantity,REGDN_Number of Bids,REGDN_Weighted Avg Price,REGUP_Unweighted Average Price,REGUP_Max Price,REGUP_Min Price,REGUP_Total Quantity,REGUP_Number of Bids,REGUP_Weighted Avg Price,RRSGN_Unweighted Average Price,RRSGN_Max Price,RRSGN_Min Price,RRSGN_Total Quantity,RRSGN_Number of Bids,RRSGN_Weighted Avg Price,RRSNC_Unweighted Average Price,RRSNC_Max Price,RRSNC_Min Price,RRSNC_Total Quantity,RRSNC_Number of Bids,RRSNC_Weighted Avg Price,NSPIN_Quantity,REGDN_Quantity,REGUP_Quantity,RRS_Quantity,price_DAH_busavg,price_DAH_houston,price_DAH_hubavg,price_DAH_N,price_DAH_S,price_DAH_W,price_RT15_busavg,price_RT15_houston,price_RT15_hubavg,price_RT15_N,price_RT15_S,price_RT15_W,price_DAH_Down Regulation,vol_DAH_Down Regulation,price_DAH_Up Regulation,vol_DAH_Up Regulation,price_DAH_Non-Spinning Reserve,vol_DAH_Non-Spinning Reserve,price_DAH_Responsive Reserve,vol_DAH_Responsive Reserve,Coal,GT,NGCC,Nuclear.x,ST,Other,Load.MW,Total,UPV,solar_ISO,wind,load_naked,load_net,ramp,ramp_abs,max_ramp,max_ramp_abs,Biomass,Hydro,Imports,Nuclear.y,Solar,Wind,ng_price,Coast,East,FarWest,North,NorthCentral,SouthCentral,Southern,West
0,2010-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15608.14,746.96,8705.99,5171.0,383.60,NaN,32308.56,0.0,0.0,0.0,1285.23,32308.56,31023.33,-131.67,131.67,2437.34,2437.34,NaN,12.31,NaN,5171.69,NaN,1285.23,5.741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15532.08,746.95,8617.26,5171.0,375.78,NaN,32520.76,0.0,0.0,0.0,1898.73,32520.76,30622.03,-401.30,401.30,2437.34,2437.34,NaN,12.32,NaN,5170.11,NaN,1898.73,5.741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15694.58,745.00,8818.14,5171.0,472.17,NaN,33190.19,0.0,0.0,0.0,2526.21,33190.19,30663.98,41.95,41.95,2437.34,2437.34,NaN,12.32,NaN,5170.15,NaN,2526.21,5.741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-01 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15502.46,744.03,9276.63,5171.0,581.74,NaN,34377.56,0.0,0.0,0.0,2961.25,34377.56,31416.31,752.33,752.33,2437.34,2437.34,NaN,12.58,NaN,5170.77,NaN,2961.25,5.741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-01-01 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15548.02,762.51,10251.94,5171.0,573.46,NaN,35592.75,0.0,0.0,0.0,2781.14,35592.75,32811.61,1395.30,1395.30,2437.34,2437.34,NaN,92.11,NaN,5170.57,NaN,2781.14,5.741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_csv("merged_df.csv") #180 MB...worth saving? Pickle instead to keep datetime features?